<a href="https://colab.research.google.com/github/iksnn/Learn-Basic-Deep-Learning/blob/main/Build_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AGENT  
By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to finish.



In [ ]:
!pip  install -qq langchain-google-genai google-generativeai

In [ ]:
from langchain.chains import ConversationChain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal, List, Dict, Optional
from pydantic import Field
from datetime import datetime

import getpass

API_KEY = getpass.getpass('Write your API KEY here:')

Write your API KEY here:··········


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    api_key=API_KEY
    )

#Exercise  
We will make Agent as a waiter in a restaurant. The Agent must have the capability to serve customer, answer question, and calculate the bill.

## Define Tools  
*   get_current_info
*   list_menu  
*   list_promo  



In [ ]:
#define timezone
from pytz import timezone
tz = timezone('Asia/Jakarta')


In [ ]:
@tool
def get_current_info() -> str:
  """Returns information about the current day."""
  return datetime.now(tz).strftime("%A")

@tool
def list_menu() -> dict:
  """Returns information about the meals and beverages menu with prices."""
  menus = {
      "Breakfast Toast": 2.0,
      "Pizza": 15.0,
      "Spaghetti": 9.0,
      "Rice Bowl": 10.0,
      "Fried Chicken": 7.0,
      "Latte": 2.0,
      "Cappuccino": 4.0,
      "Mineral Water": 3.0,
      "Juice": 5.0,
      "Soda": 7.0
  }
  return menus

@tool
def list_promo(day:str) -> dict:
  """Returns the list of daily promos."""
  promo = {
    "Sunday":{ "item": "Juice", "discount": "10%"},
    "Monday":{ "item": "Pizza", "discount": "10%"},
    "Tuesday":{ "item": "Rice Bowl", "discount": "5%"},
    "Wednesday":{ "item": "Cappuccino", "discount": "50%"},
    "Thursday":{ "item": "Soda", "discount": "10%"},
    "Friday":{ "item": "Spaghetti", "discount": "15%"},
    "Saturday":{ "item": "Pizza", "discount": "15%"}
  }
  return promo[day]

In [ ]:
tools = [get_current_info, list_menu, list_promo]

#Define Agent  
*  Make prompt for giving instruction to LLM  
*  Add tools description in prompt
*  Use AgentExecutor for build Agent with defined tools and LLM.

In [ ]:
system_prompt = '''
You're a professional waiter in a restaurant. You know all the menu and promotions. Answer questions about the menu, price, and promotions. Don't answer any questions outside those contexts.  Do not use your own knowledge to answer the question; use the tools provided.

You are provided some tools:
- `get_current_info()` tool gets information about the current date and day. Example: `"Monday`.
- `list_promo()` tool gets information about daily promos.  Example output: `{{"item": "Spaghetti", "discount": "15%"}}`
- `list_menu()` tool gets price information about meals and beverages with price. Example output: `{{"Fried Chicken": 7.0,"Latte": 2.0}}`


Note : For total price, don't forget to calculate promo if any.
'''

In [ ]:
prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", "Question: {input}\nHistory Chat: {history}"),
            ("placeholder", "{agent_scratchpad}"),
        ])

In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
question = "What promo for today?"
history = []

## Try out Agent with chat history
Scenario:  
*  Ask promo for today  
*  Chose the promo and one non-promo menu  


Goals : Agent can make correct decision to chose tool and give correct responses to customer

In [ ]:
try:
  response = agent_executor.invoke(
      {
          "input": question,
          "history": history
      }
  )
  print("\n",response['output'])
except Exception as e:
  error = str(e)
  print("\nError:",e)



> Entering new AgentExecutor chain...

Invoking: `get_current_info` with `{}`


Tuesday
Invoking: `list_promo` with `{'day': 'Tuesday'}`


{'item': 'Rice Bowl', 'discount': '5%'}Today's promo is 5% discount on Rice Bowl.

> Finished chain.

 Today's promo is 5% discount on Rice Bowl.


In [ ]:
history.append(("human",f"{question}"))
history.append(("ai",f"{response['output']}"))

In [ ]:
history

[('human', 'What promo for today?'),
 ('ai', "Today's promo is 5% discount on Rice Bowl.")]

In [ ]:
question = "Okay, i want to see all menu"

In [ ]:
try:
  response = agent_executor.invoke(
      {
          "input": question,
          "history": history
      }
  )
  print("\n",response['output'])
except Exception as e:
  error = str(e)
  print("\nError:",e)



> Entering new AgentExecutor chain...

Invoking: `list_menu` with `{}`


{'Breakfast Toast': 2.0, 'Pizza': 15.0, 'Spaghetti': 9.0, 'Rice Bowl': 10.0, 'Fried Chicken': 7.0, 'Latte': 2.0, 'Cappuccino': 4.0, 'Mineral Water': 3.0, 'Juice': 5.0, 'Soda': 7.0}Here is the menu: Breakfast Toast($2), Cappuccino($4), Fried Chicken($7), Juice($5), Latte($2), Mineral Water($3), Pizza($15), Rice Bowl($10), Soda($7), Spaghetti($9).


> Finished chain.

 Here is the menu: Breakfast Toast($2), Cappuccino($4), Fried Chicken($7), Juice($5), Latte($2), Mineral Water($3), Pizza($15), Rice Bowl($10), Soda($7), Spaghetti($9).



In [ ]:
history.append(("human",f"{question}"))
history.append(("ai",f"{response['output']}"))

In [ ]:
question = "I want to order rice bowl and latte"

In [ ]:
try:
  response = agent_executor.invoke(
      {
          "input": question,
          "history": history
      }
  )
  print("\n",response['output'])
except Exception as e:
  error = str(e)
  print("\nError:",e)



> Entering new AgentExecutor chain...

Invoking: `list_menu` with `{}`


{'Breakfast Toast': 2.0, 'Pizza': 15.0, 'Spaghetti': 9.0, 'Rice Bowl': 10.0, 'Fried Chicken': 7.0, 'Latte': 2.0, 'Cappuccino': 4.0, 'Mineral Water': 3.0, 'Juice': 5.0, 'Soda': 7.0}Rice Bowl is $10 and Latte is $2.  The total is $12.  Today's promo is 5% off on Rice Bowl, so the total will be $11.4.


> Finished chain.

 Rice Bowl is $10 and Latte is $2.  The total is $12.  Today's promo is 5% off on Rice Bowl, so the total will be $11.4.

